In [1]:
from pathlib import Path
import tflite
import tvm
from tvm import relay, transform
from tvm import te
import matplotlib.pyplot as plt
import imageio
import cv2
import numpy as np
from PIL import Image

In [2]:
model_path = Path.cwd().parent / 'models' / 'selfie_segmentation' / 'selfie_segmentation.tflite'
model_path.exists()

True

In [5]:
with open(model_path, 'rb') as f:
    tflite_model_buf = f.read()
    tflite_model = tflite.Model.GetRootAsModel(tflite_model_buf, 0)
    input_tensor = "input"
    input_shape  = (1, 256, 256, 3)
    input_dtype  = "float32"
        
    mod, params = relay.frontend.from_tflite(
        tflite_model, 
        shape_dict = {input_tensor: input_shape},
        dtype_dict = {input_tensor: input_dtype}
    )
    target = "llvm"
    
    desired_layouts = {'nn.conv2d': ['NCHW', 'default']}

    # Convert the layout to NCHW
    seq = tvm.transform.Sequential([relay.transform.RemoveUnusedFunctions(),
                                    relay.transform.ConvertLayout(desired_layouts)])
    
    with transform.PassContext(opt_level=3):
        mod = seq(mod)
        lib=relay.build(mod, target, params=params)
        lib.export_library(str(model_path.with_suffix('.so')))

NotImplementedError: Custom operators are currently not supported